In [ ]:
-- instance Monoid FList a where


-- instance Semigroup (FList a) where
class MyFunctor f where
    fmap :: (a->b) -> f a -> f b

class MyMonoid m where
    mempty :: m
    mappend :: m -> m -> m

instance MyFunctor Maybe where
    fmap :: (a->b) -> Maybe a -> Maybe b
    fmap f (Just a) = Just (f a)
    fmap f _ = Nothing

instance MyFunctor [] where
    fmap :: (a -> b) -> [a] -> [b]
    fmap f (x:xs) = f x : fmap f xs
    fmap f _ = []


instance MyMonoid [a] where
    mempty = []
    mappend = (++)

instance MyMonoid Int where
    mempty = 0
    mappend = (+)

-- data Maybe a = Just a | Nothing
-- :i Monoid